<a href="https://colab.research.google.com/github/tanmaybohra/Appliances-Energy-Prediction/blob/main/Appliances_CatBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path = ('/content/drive/MyDrive/Copy of data_application_energy.csv')
df = pd.read_csv(file_path)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19735 entries, 0 to 19734
Data columns (total 29 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         19735 non-null  object 
 1   Appliances   19735 non-null  int64  
 2   lights       19735 non-null  int64  
 3   T1           19735 non-null  float64
 4   RH_1         19735 non-null  float64
 5   T2           19735 non-null  float64
 6   RH_2         19735 non-null  float64
 7   T3           19735 non-null  float64
 8   RH_3         19735 non-null  float64
 9   T4           19735 non-null  float64
 10  RH_4         19735 non-null  float64
 11  T5           19735 non-null  float64
 12  RH_5         19735 non-null  float64
 13  T6           19735 non-null  float64
 14  RH_6         19735 non-null  float64
 15  T7           19735 non-null  float64
 16  RH_7         19735 non-null  float64
 17  T8           19735 non-null  float64
 18  RH_8         19735 non-null  float64
 19  T9  

In [ ]:
from datetime import datetime
import time
import dateutil

In [ ]:
# Converting to day of the week
df["days"]=(df['date'].str.split(' ').str[0])
df["days"]=(df['days'].apply(dateutil.parser.parse, dayfirst=True))
df["days"]=(df['days'].dt.day_name())

In [ ]:
df.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2,days
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,17.166667,55.20,7.026667,84.256667,17.200000,41.626667,18.2,48.900000,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433,Tuesday
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,17.166667,55.20,6.833333,84.063333,17.200000,41.560000,18.2,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195,Tuesday
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,17.166667,55.09,6.560000,83.156667,17.200000,41.433333,18.2,48.730000,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668,Tuesday
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,17.166667,55.09,6.433333,83.423333,17.133333,41.290000,18.1,48.590000,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389,Tuesday
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,17.200000,55.09,6.366667,84.893333,17.200000,41.230000,18.1,48.590000,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097,Tuesday


In [ ]:
df_dateNappli = pd.DataFrame({'date': df['date'], 'Appliances': df['Appliances']})
df_dateNappli.date=df_dateNappli.date.astype('datetime64[ns]')
df_dateNappli.set_index('date', inplace=True)

In [ ]:
df['hour'] = pd.to_datetime(df['date']).dt.hour

In [ ]:
df.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2,days,hour
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,17.166667,55.20,7.026667,84.256667,17.200000,41.626667,18.2,48.900000,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433,Tuesday,17
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,17.166667,55.20,6.833333,84.063333,17.200000,41.560000,18.2,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195,Tuesday,17
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,17.166667,55.09,6.560000,83.156667,17.200000,41.433333,18.2,48.730000,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668,Tuesday,17
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,17.166667,55.09,6.433333,83.423333,17.133333,41.290000,18.1,48.590000,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389,Tuesday,17
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,17.200000,55.09,6.366667,84.893333,17.200000,41.230000,18.1,48.590000,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097,Tuesday,17


In [ ]:
energy = df.copy()

In [ ]:
energy.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2,days,hour
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,17.166667,55.20,7.026667,84.256667,17.200000,41.626667,18.2,48.900000,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433,Tuesday,17
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,17.166667,55.20,6.833333,84.063333,17.200000,41.560000,18.2,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195,Tuesday,17
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,17.166667,55.09,6.560000,83.156667,17.200000,41.433333,18.2,48.730000,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668,Tuesday,17
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,17.166667,55.09,6.433333,83.423333,17.133333,41.290000,18.1,48.590000,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389,Tuesday,17
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,17.200000,55.09,6.366667,84.893333,17.200000,41.230000,18.1,48.590000,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097,Tuesday,17


In [ ]:
# Let us drop rest columns
energy.drop(df.iloc[:, 2:28], inplace = True, axis = 1)

In [ ]:
energy.head()

,date,Appliances,rv2,days,hour
0,2016-01-11 17:00:00,60,13.275433,Tuesday,17
1,2016-01-11 17:10:00,60,18.606195,Tuesday,17
2,2016-01-11 17:20:00,50,28.642668,Tuesday,17
3,2016-01-11 17:30:00,50,45.410389,Tuesday,17
4,2016-01-11 17:40:00,60,10.084097,Tuesday,17


In [ ]:
energy.drop(['date'], inplace = True, axis = 1)

In [ ]:
energy.drop(['rv2'], inplace = True, axis = 1)

In [ ]:
energy.head()

,Appliances,days,hour
0,60,Tuesday,17
1,60,Tuesday,17
2,50,Tuesday,17
3,50,Tuesday,17
4,60,Tuesday,17


In [ ]:
energy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19735 entries, 0 to 19734
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Appliances  19735 non-null  int64 
 1   days        19735 non-null  object
 2   hour        19735 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 462.7+ KB


In [ ]:
energy.days.unique()

array(['Tuesday', 'Thursday', 'Wednesday', 'Friday', 'Saturday', 'Sunday',
       'Monday'], dtype=object)

In [ ]:
!pip install catboost

     |████████████████████████████████| 76.3 MB 63 kB/s 


In [ ]:
# importing required libraries
from catboost import CatBoostRegressor, Pool, FeaturesData
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score,accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
import time
from math import sqrt
from sklearn import preprocessing, model_selection, metrics

In [84]:
x = energy[['days','hour']]
y = energy['Appliances']

In [85]:
categorical_var = ['days']

In [86]:
train_x, test_x, train_y, test_y = train_test_split(x,y,test_size=0.25,random_state=40)

In [87]:
train_dataset = Pool(train_x,train_y,cat_features=categorical_var)
eval_dataset = Pool(test_x,test_y,cat_features=categorical_var)

In [88]:
model = CatBoostRegressor(iterations = 1600,
                          learning_rate = 0.05,
                          max_depth = 7,
                          loss_function = 'RMSE',
                          random_seed = 10,
                          eval_metric = 'R2')

model.fit(train_dataset,
        eval_set = eval_dataset,
        plot = True,
        verbose = 0)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [89]:
model.get_best_score()

{'learn': {'R2': 0.2098502182425218, 'RMSE': 91.49024538935414},
 'validation': {'R2': 0.1482153913369062, 'RMSE': 93.495721805167}}

In [90]:
model = CatBoostRegressor(iterations = 1600,
                          learning_rate = 0.05,
                          loss_function = 'RMSE',
                          random_seed = 10,
                          eval_metric = 'R2')


#Declare parameters to tune and values to try
grid = {'learning_rate': [0.03, 0.1],
        'depth': [4, 6, 7, 10],
        'l2_leaf_reg': [1, 3, 5,]}

# #Find optimum parameters
model.grid_search(grid,train_dataset,plot=True)

model.fit(train_dataset,
          eval_set = eval_dataset,
          plot = True,
          verbose = 0)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Streaming output truncated to the last 5000 lines.
1421:	learn: 0.2917981	test: 0.1590440	best: 0.1805858 (189)	total: 16.2s	remaining: 2.03s
1422:	learn: 0.2918059	test: 0.1590476	best: 0.1805858 (189)	total: 16.2s	remaining: 2.02s
1423:	learn: 0.2918072	test: 0.1590484	best: 0.1805858 (189)	total: 16.2s	remaining: 2s
1424:	learn: 0.2918111	test: 0.1590487	best: 0.1805858 (189)	total: 16.2s	remaining: 1.99s
1425:	learn: 0.2918409	test: 0.1589743	best: 0.1805858 (189)	total: 16.3s	remaining: 1.98s
1426:	learn: 0.2918482	test: 0.1589747	best: 0.1805858 (189)	total: 16.3s	remaining: 1.97s
1427:	learn: 0.2920385	test: 0.1589860	best: 0.1805858 (189)	total: 16.3s	remaining: 1.96s
1428:	learn: 0.2921431	test: 0.1589430	best: 0.1805858 (189)	total: 16.3s	remaining: 1.95s
1429:	learn: 0.2923142	test: 0.1587913	best: 0.1805858 (189)	total: 16.3s	remaining: 1.94s
1430:	learn: 0.2923354	test: 0.1588040	best: 0.1805858 (189)	total: 16.3s	remaining: 1.93s
1431:	learn: 0.2923412	test: 0.1587453	bes

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [91]:
model.get_best_score()

{'learn': {'R2': 0.29314609665360936, 'RMSE': 86.53362764051114},
 'validation': {'R2': 0.14668541951651048, 'RMSE': 93.57965243815171}}